In [ ]:
def concatSampleHalves(sample1, sample2):
    
    concatenatedData = anndata.concat([sample1, sample2], join="outer")
    
    # Re-Clustering Half2_B_ii
    # PCA
    sc.tl.pca(concatenatedData, svd_solver='arpack')

    # Neighbors and UMAP Computations
    sc.pp.neighbors(concatenatedData, n_neighbors=50, n_pcs =50, use_rep = "Scanorama")
    sc.tl.umap(concatenatedData) 

    # Re-calculating cluster numbers 
    sc.tl.leiden(concatenatedData, resolution = 0.3, key_added = "leiden_0.3")

    # Calculates nuclei and gene information specific to the sample object 
    sc.pp.calculate_qc_metrics(concatenatedData, percent_top=None, log1p=False, inplace=True)
    
    return concatenatedData

In [ ]:
def figure7(fullSampleObj, halfSampleObj, fourthSampleObj, sampleInfo, savePath):
    # Create a figure with subplots
    fig = plt.figure(figsize=(20,25), dpi=150)

    row = 3 
    col = 6

    ax1 = plt.subplot2grid((row, col), (0, 0), colspan=2) # Full Sample UMAP
    ax2 = plt.subplot2grid((row, col), (1, 0), colspan=2) # 1/2 Sample UMAP
    ax3 = plt.subplot2grid((row, col), (2, 0), colspan=2) # 1/4 Sample UMAP
#     ax4 = plt.subplot2grid((row, col), (3, 0), colspan=2) # 1/8 Sample UMAP

    ax5 = plt.subplot2grid((row, col), (0, 2), colspan=2) # Full Sample DE Sample1 
    ax6 = plt.subplot2grid((row, col), (0, 4), colspan=2) # Full Sample DE Sample2

    ax7 = plt.subplot2grid((row, col), (1, 2), colspan=2) # 1/2 Sample DE Sample1 
    ax8 = plt.subplot2grid((row, col), (1, 4), colspan=2) # 1/2 Sample DE Sample2

    ax9 = plt.subplot2grid((row, col), (2, 2), colspan=2) # 1/4 Sample DE Sample1 
    ax10 = plt.subplot2grid((row, col), (2, 4), colspan=2) # 1/4 Sample DE Sample2

#     ax11 = plt.subplot2grid((row, col), (3, 2), colspan=2) # 1/8 Sample DE Sample1 
#     ax12 = plt.subplot2grid((row, col), (3, 4), colspan=2) # 1/8 Sample DE Sample2

    # Getting Sample Information 
    condition = sampleInfo[0]
    sample1_name = sampleInfo[1]
    sample2_name = sampleInfo[2]
    
    sample1_mouseSex = mouseSex(sample1_name)
    sample2_mouseSex = mouseSex(sample2_name)
    
    half = sampleInfo[3]
    quarter = sampleInfo[4]
#     eith = sampleInfo[5]
    
    # Sample UMAPS
    sc.pl.umap(fullSampleObj, color='leiden_0.3', legend_loc='on data', show=False, ax=ax1)
    fullSampleNuc = fullSampleObj.n_obs
    ax1.set_title(f'Full Sample UMAP ({fullSampleNuc} Nuclei)')
    
    sc.pl.umap(halfSampleObj, color='leiden_0.3', legend_loc='on data', show=False, ax=ax2)
    halfSampleNuc = halfSampleObj.n_obs
    ax2.set_title(f'{half} Sample UMAP ({halfSampleNuc} Nuclei)')
    
    sc.pl.umap(fourthSampleObj, color='leiden_0.3', legend_loc='on data', show=False, ax=ax3)
    quartSampleNuc = fourthSampleObj.n_obs
    ax3.set_title(f'Quarter {quarter} Sample UMAP ({quartSampleNuc} Nuclei)')
    
#     sc.pl.umap(eithSampleObj, color='leiden_0.3', legend_loc='on data', show=False, ax=ax4)
#     eithSampleNuc = eithSampleObj.n_obs
#     ax4.set_title(f'Eith {eith} Sample UMAP ({eithSampleNuc} Nuclei)')
 
    # Full Sample DE
    fullSampleObj.uns['log1p'] = {'base': None}
    sc.tl.rank_genes_groups(fullSampleObj, groupby='sample', method='wilcoxon')
    deGraph(fullSampleObj, sample1_name, sample2_name, 15, ax5)
    deGraph(fullSampleObj, sample2_name, sample1_name, 15, ax6)
    
    # 1/2 Sample DE
    halfSampleObj.uns['log1p'] = {'base': None}
    sc.tl.rank_genes_groups(halfSampleObj, groupby='sample', method='wilcoxon')
    deGraph(halfSampleObj, sample1_name, sample2_name, 15, ax7)
    deGraph(halfSampleObj, sample2_name, sample1_name, 15, ax8)

    # 1/4 Sample DE
    fourthSampleObj.uns['log1p'] = {'base': None}
    sc.tl.rank_genes_groups(fourthSampleObj, groupby='sample', method='wilcoxon')
    deGraph(fourthSampleObj, sample1_name, sample2_name, 15, ax9)
    deGraph(fourthSampleObj, sample2_name, sample1_name, 15, ax10)
    
    # 1/8 Sample DE
#     sc.tl.rank_genes_groups(eithSampleObj, groupby='sample', method='wilcoxon')
#     deGraph(eithSampleObj, sample1_name, sample2_name, 15, ax11)
#     deGraph(eithSampleObj, sample2_name, sample1_name, 15, ax12)
    
    fig.suptitle(f'{condition} - Sample {sample1_name} ({sample1_mouseSex}) vs. {sample2_name} ({sample2_mouseSex})', 
                 fontsize=20, fontweight='bold', y=0.92)
    
    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    
    # Saving the figure onto the computer 
    plt.savefig(os.path.join(savePath, f'{sample1_name}_{sample2_name}_{half}_{quarter}.jpg'), bbox_inches = 'tight')

In [ ]:
def plotFigure7(comparLst, sampleFolder, savePath):
    
    for lst in comparLst:
        
        condition = lst[0]
        sample1_name = lst[1]
        sample2_name = lst[2]
        
        fullSampleObj = sc.read(f'F:/SampleData/{sampleFolder}/{condition}_{sample1_name}_{sample2_name}')
        
        for half in ['Half1', 'Half2']:
            half1_path = search_files('F:/SampleData/SampleHalves/', f'{sample1_name}_{half}')
            half2_path = search_files('F:/SampleData/SampleHalves/', f'{sample2_name}_{half}')
            half1_obj = sc.read(half1_path[0])
            half2_obj = sc.read(half2_path[0])
            
            halfConcat = concatSampleHalves(half1_obj, half2_obj)
            
            for quarter in ['A', 'B']:
                fourth1_path = search_files('F:/SampleData/SampleQuarters/', f'{sample1_name}_{half}_{quarter}')
                fourth2_path = search_files('F:/SampleData/SampleQuarters/', f'{sample2_name}_{half}_{quarter}')
                fourth1_obj = sc.read(fourth1_path[0])
                fourth2_obj = sc.read(fourth2_path[0])
                
                fourthConcat = concatSampleHalves(fourth1_obj, fourth2_obj)
                
                figure7(fullSampleObj, halfConcat, fourthConcat, [condition, sample1_name, sample2_name, half, quarter], 
                        savePath)
                
#                 for eith in ['i', 'ii']:
#                     eith1_path = search_files('F:/SampleData/SampleEiths/', f'{sample1_name}_{half}_{quarter}_{eith}')
#                     eith2_path = search_files('F:/SampleData/SampleEiths/', f'{sample2_name}_{half}_{quarter}_{eith}')
#                     eith1_obj = sc.read(eith1_path[0])
#                     eith2_obj = sc.read(eith2_path[0])

#                     eithConcat = concatSampleHalves(eith1_obj, eith2_obj)
                        
#                     figure7(fullSampleObj, halfConcat, fourthConcat, eithConcat, [condition, sample1_name, sample2_name, 
#                             half, quarter, eith], savePath)